In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from tensorflow.keras.callbacks import TensorBoard
import time

def configure_model(layers_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],debugflag=False):
    if debugflag:
        model = tf.keras.Sequential([
            layers.Dense(64, activation='linear', input_shape=(dset_features.shape[1],)),  # Hidden layer 1
            layers.Dense(32, activation='selu'),  # Hidden layer 2
            layers.Dense(1, activation='linear')  # Output layer
            ])
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        return model
        
    model = tf.keras.Sequential()

    for layer_config in layers_config:
        layer_type = layer_config[0]

        if layer_type == 'dense':
            neurons, activation = layer_config[1], layer_config[2]
            if len(layer_config) > 3 and layer_config[3] == 'input':
                input_shape = layer_config[4]
                model.add(layers.Dense(neurons, activation=activation, input_shape=(input_shape,)))
            else:
                model.add(layers.Dense(neurons, activation=activation))
        elif layer_type == 'dropout':
            rate = layer_config[1]
            model.add(layers.Dropout(rate))
        elif layer_type == 'activation':
            activation = layer_config[1]
            model.add(layers.Activation(activation))
        elif layer_type == 'conv2d':
            filters, kernel_size, strides, activation = layer_config[1], layer_config[2], layer_config[3], layer_config[4]
            model.add(layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, activation=activation))
        elif layer_type == 'maxpool2d':
            pool_size = layer_config[1]
            model.add(layers.MaxPool2D(pool_size=pool_size))
        elif layer_type == 'rnn':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.SimpleRNN(units, return_sequences=return_sequences))
        elif layer_type == 'lstm':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.LSTM(units, return_sequences=return_sequences))
        elif layer_type == 'gru':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.GRU(units, return_sequences=return_sequences))
        elif layer_type == 'attention':
            model.add(layers.MultiHeadAttention(num_heads=layer_config[1], key_dim=layer_config[2]))
        elif layer_type == 'cbam':
            model.add(CBAM())
        elif layer_type == 'channel_attention':
            model.add(ChannelAttention())
        elif layer_type == 'linear_attention':
            model.add(LinearAttention())
        elif layer_type == 'nonlinear_attention':
            model.add(NonLinearAttention())

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model


def train_model(model, X_train, y_train, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2", total_epoch=50):
    tensorboard_callback = TensorBoard(log_dir='./../../Observation/'+model_name+'/'+model_name+dataset_name+'logs')

    start_time = time.time()

    history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])

    training_time = time.time() - start_time

    predictions = model.predict(X_test)

    return model, history, predictions, training_time

def test_model(model, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2",threshold=0.5):
    start_time = time.time()
    predictions = model.predict(X_test)
    inference_time = time.time() - start_time
    
    metrics = calculate_metrics(predictions, y_test,threshold)
    metrics['Training Time'] = training_time
    metrics['Inference Time'] = inference_time
    metrics['Model type'] = model_name
    metrics['dataset'] = dataset_name

    print("Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    metrics_list = list(metrics.values())
    pd.DataFrame([metrics_list], columns=metrics.keys()).to_csv('./../../Observation/'+model_name+'/'+model_name+dataset_name+'metrics.csv', index=False)

    return metrics

def calculate_metrics(predictions, true_labels, threshold=0.5):
    #threshold = 0.5  # Define the threshold
    correct_predictions = np.sum(np.abs(true_labels - predictions.flatten()) <= threshold)
    total_predictions = len(true_labels)
    accuracy = correct_predictions / total_predictions  # Final accuracy
    mse = mean_squared_error(true_labels, predictions)  # Mean Squared Error
    mae = mean_absolute_error(true_labels, predictions)  # Mean Absolute Error
    rmse = np.sqrt(mse)  # Root Mean Squared Error
    f1 = f1_score(true_labels.round(), predictions.round(), average='micro')  # F1 Score

    return {
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'F1 Score': f1
    }

In [6]:
# Load dataset
dataset_name = "addition_dataset_2"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
#perceptron_config = [(1, 'dense')]
mlp_config = [
    ('dense', 64, 'linear', 'input', dset_features.shape[1]),
    ('dense', 32, 'selu'),
    ('dense', 1, 'linear')
]
mlp_config = [
    ('dense', 3, 'linear', 'input', dset_features.shape[1]),
    ('dense', 6, 'relu'),
    ('dense', 2, 'relu'),
    ('dense', 1, 'linear')
]
model = configure_model(mlp_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],debugflag=False)

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: 481413152.0000 - val_accuracy: 2.5000e-04 - val_loss: 1.4088
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 1.3338 - val_accuracy: 2.5000e-04 - val_loss: 1.3082
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 1.3239 - val_accuracy: 2.5000e-04 - val_loss: 1.3076
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 1.3419 - val_accuracy: 2.5000e-04 - val_loss: 1.3068
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 1.3308 - val_accuracy: 2.5000e-04 - val_loss: 1.3015
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 1.3218 - val_accuracy: 2.5000e-04 - val_loss: 1.3038
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 1.3573 - val_accuracy: 2.5000e-04 - val_loss: 1.2927
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

In [8]:
# Load dataset
dataset_name = "simple_sin_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
#perceptron_config = [(1, 'dense')]
mlp_config = [
    ('dense', 8, 'relu', 'input', dset_features.shape[1]),
    ('dense', 6, 'relu'),
    ('dense', 3, 'relu'),
    ('dense', 1, 'linear')
]
model = configure_model(mlp_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='multi_layer_perceptron',total_epoch=50,dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,dataset_name=dataset_name,model_name='multi_layer_perceptron',threshold=0.9)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0000e+00 - loss: 2128811.5000 - val_accuracy: 2.5000e-04 - val_loss: 13249.8789
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 7985.8442 - val_accuracy: 2.5000e-04 - val_loss: 1491.3162
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 1066.1755 - val_accuracy: 2.5000e-04 - val_loss: 357.1552
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 276.7928 - val_accuracy: 2.5000e-04 - val_loss: 113.7858
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 90.3148 - val_accuracy: 2.5000e-04 - val_loss: 40.2912
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 32.0452 - val_accuracy: 2.5000e-04 - val_loss: 14.2867
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: 11.4246 - val_accuracy: 2.5000e-04 - val_loss: 4.7982
Epoch 8/50
500/500 ━━━━━━━━━━

In [9]:
# Load dataset
dataset_name = "simple_cos_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
#perceptron_config = [(1, 'dense')]
mlp_config = [
    ('dense', 8, 'relu', 'input', dset_features.shape[1]),
    ('dense', 6, 'relu'),
    ('dense', 3, 'relu'),
    ('dense', 1, 'linear')
]
model = configure_model(mlp_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='multi_layer_perceptron',total_epoch=50,dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,dataset_name=dataset_name,model_name='multi_layer_perceptron',threshold=0.9)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0000e+00 - loss: 647.2635 - val_accuracy: 0.0000e+00 - val_loss: 0.4975
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 0.5015 - val_accuracy: 0.0000e+00 - val_loss: 0.4981
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 0.5024 - val_accuracy: 0.0000e+00 - val_loss: 0.4960
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 0.5048 - val_accuracy: 0.0000e+00 - val_loss: 0.5266
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 0.5038 - val_accuracy: 0.0000e+00 - val_loss: 0.4979
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 0.5065 - val_accuracy: 0.0000e+00 - val_loss: 0.4980
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 0.5139 - val_accuracy: 0.0000e+00 - val_loss: 0.4987
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc

In [12]:
# Load dataset
dataset_name = "substraction_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
#perceptron_config = [(1, 'dense')]
mlp_config = [
    ('dense', 64, 'linear', 'input', dset_features.shape[1]),
    ('dense', 32, 'selu'),
    ('dense', 1, 'linear')
]
mlp_config = [
    ('dense', 3, 'linear', 'input', dset_features.shape[1]),
    ('dense', 6, 'relu'),
    ('dense', 2, 'relu'),
    ('dense', 1, 'linear')
]
model = configure_model(mlp_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],debugflag=True)

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 6.0050e-05 - loss: 13228188.0000 - val_accuracy: 2.5000e-04 - val_loss: 2.6897
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 2.4151e-05 - loss: 2.7073 - val_accuracy: 2.5000e-04 - val_loss: 2.6594
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 1.2150e-04 - loss: 2.7643 - val_accuracy: 2.5000e-04 - val_loss: 2.6464
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 4.1130e-06 - loss: 2.6970 - val_accuracy: 2.5000e-04 - val_loss: 2.6275
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 4.0379e-05 - loss: 2.6502 - val_accuracy: 2.5000e-04 - val_loss: 2.5144
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 9.1050e-05 - loss: 2.6267 - val_accuracy: 2.5000e-04 - val_loss: 2.9362
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 2.3968e-05 - loss: 2.5274 - val_accuracy: 2.5000e-04 - val_loss: 2.4258
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step 

In [13]:
# Load dataset
dataset_name = "doubling_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
#perceptron_config = [(1, 'dense')]

mlp_config = [
    ('dense', 3, 'linear', 'input', dset_features.shape[1]),
    ('dense', 6, 'relu'),
    ('dense', 2, 'relu'),
    ('dense', 1, 'linear')
]
model = configure_model(mlp_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],debugflag=True)

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0000e+00 - loss: 60435236.0000 - val_accuracy: 0.0000e+00 - val_loss: 26.9355
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: 9.7074 - val_accuracy: 0.0000e+00 - val_loss: 8.6100
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 8.6249 - val_accuracy: 0.0000e+00 - val_loss: 8.5611
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 8.7294 - val_accuracy: 0.0000e+00 - val_loss: 8.6797
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 8.6514 - val_accuracy: 2.5000e-04 - val_loss: 8.4656
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 8.5430 - val_accuracy: 2.5000e-04 - val_loss: 8.4286
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 8.3908 - val_accuracy: 2.5000e-04 - val_loss: 8.0060
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

In [5]:
# Load dataset
dataset_name = "multiplication_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
#perceptron_config = [(1, 'dense')]

mlp_config = [
    ('dense', 10, 'linear', 'input', dset_features.shape[1]),
    ('dense', 6, 'relu'),
    ('dense', 2, 'relu'),
    ('dense', 1, 'linear')
]
model = configure_model(mlp_config, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['accuracy'],debugflag=False)

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name,total_epoch=150)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

Epoch 1/150


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 31942395482341376.0000 - val_accuracy: 0.0000e+00 - val_loss: 30399538002919424.0000
Epoch 2/150
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 29740290587754496.0000 - val_accuracy: 0.0000e+00 - val_loss: 20758889054076928.0000
Epoch 3/150
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 16953341871390720.0000 - val_accuracy: 0.0000e+00 - val_loss: 5263451282735104.0000
Epoch 4/150
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: 3984535918215168.0000 - val_accuracy: 0.0000e+00 - val_loss: 2025107815923712.0000
Epoch 5/150
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 2022711626825728.0000 - val_accuracy: 0.0000e+00 - val_loss: 1998691216916480.0000
Epoch 6/150
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 2018558930321408.0000 - val_accuracy: 0.0000e+00 - val_loss: 1998748527886336.0000
Epoch

In [9]:
# Load dataset
dataset_name = "natural_log_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
#perceptron_config = [(1, 'dense')]

mlp_config = [
    ('dense', 10, 'linear', 'input', dset_features.shape[1]),
    ('dense', 6, 'relu'),
    ('dense', 2, 'relu'),
    ('dense', 1, 'linear')
]
model = configure_model(mlp_config, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['accuracy'],debugflag=False)

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name,total_epoch=150)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='multi_layer_perceptron',dataset_name=dataset_name)

c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.0000e+00 - loss: 57818.7930 - val_accuracy: 2.5000e-04 - val_loss: 79.4324
Epoch 2/150
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 79.1628 - val_accuracy: 2.5000e-04 - val_loss: 78.1897
Epoch 3/150
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 77.7473 - val_accuracy: 2.5000e-04 - val_loss: 76.3890
Epoch 4/150
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 75.7212 - val_accuracy: 2.5000e-04 - val_loss: 74.0126
Epoch 5/150
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 978us/step - accuracy: 0.0000e+00 - loss: 72.9952 - val_accuracy: 0.0000e+00 - val_loss: 71.0392
Epoch 6/150
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 965us/step - accuracy: 0.0000e+00 - loss: 70.3203 - val_accuracy: 0.0000e+00 - val_loss: 67.4655
Epoch 7/150
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 66.4372 - val_accuracy: 0.0000e+00 - val_loss: 63.3329
Epoch 8/150
500/500 